# What is GraphQL

GraphQL is **Data Query Language** developed by facebook on 2012 before it's publicly released on 2015. On 2018, the GraphQL project was moved from Facebook and then developed in GraphQL Foundation under Linux Foundation. Now, GraphQL become a new standard of API, replacing the REST method that has been on the throne for years. You can say goodbye, REST. 

![](res/graphql_logo.png)

# GraphQL vs REST

You might question, how graphql conquer REST ? all you need to know first is that REST is good before the age of mobile apps. So, what's the problem with REST ? 

## Over/Underfetching
The growth of data resulting in the increasing of the need for data use. This affect API's structure. Some companies would just remodel their REST API, and ended up in tons of [API endpoints](https://searchapparchitecture.techtarget.com/definition/API-endpoint). In other word, the main problem of REST is the API will somehow ended up with excessive amount of endpoint wich resulting in underfetching. Underfetching means each endpoints doesn't give necessary result (too litle information) so we have to call into several endpoints. On the other side, a little number of endpoints doesn't always mean better. Somehow, with little endpoints, we will ended up with too many information given from the endpoint. This will cause Overfetching.

See this illustration
![](res/fetch_rest.png)

GraphQL solve this problem by making a schema. A schema is like a contract between client and server, denoting what data the client can get. The client can get anything within the schema using a **query**. By doing so, the client will **get what they asked** See the illustratin below

![](res/fetch_graphql.png)

## Insightful Analytic
Since GraphQL accepts query as its main input, it can help the team to analyze what data is asked by users, while using simple REST could gives bias (in overfetching case).



source: howtographql.com


# How GraphQL works

Let's say, you requested a burger on a server. 


**REST**

On a casual REST, you will send HTTP request to let's say `api.get-burger/`
and get a response like below 
```JSON
"burger": {
    "top" : {
        bun : 1
    }
    "fill" : {
        "ham" : 1,
        "cheese" : 1,
        "pickles" : 4,
        "tomatoes" : 2,
        "lettuces" : 2
    }
    "base" :{
        bun : 1
    }
}
```

**GraphQL**

If you're using GraphQL, you can request using a query. Let's say, you wanted the burger with only base bun with cheese and ham on it. You send the request like this : 
```JSON
query{
    sandwich{
        fill{
            cheese
            ham 
        }
        base{
            bun
        }
    }
}
```
You will get the result as below
```JSON
"sandwich": {
    "fill" : {
        "cheese" : 1,
        "ham" : 1
    }
    "base" :{
        bun : 1
    }
}
```
See the different ? you can customize your burger order with GraphQL

Generally, query you send to GraphQL server are divided into two : 
## Query
Query works like GET in REST, to get something we asked. The only different is, we strong-typed what we requesting for in a query. By default, our query is categorized as Query, means to get the data. 

## Mutate
Creating, Updating, and Deleting instances can be done in `mutate` type of queries. 

To see full explanation of GraphQL queries, you may go to [how-to-graphql's](https://graphql.org/learn/queries/) website

# Making GraphQL API with Python

At first GraphQL was developed under **React** language. But don't worry, Python will always find its way. We have `graphene`, a python package to make GraphQL API easily. In order to build this app, we need: 
- **Flask** for app's framework
- **SQLAlchemy** for database modelling
- Chinook SQLite Database sample ([download here](https://www.sqlitetutorial.net/sqlite-sample-database/))

If you haven't familiar with Flask or SQLAlchemy, I recommend you to take the quick reading on my articles for introduction to Flask and SQLAlchemy. 

Let's back to the business. We will build our app, consisting of :
- database.py
- models.py
- schema.py
- app.py

The relation between those file will be described in this illustration:
![](res/flow.png)

the `database.py` will establish a connection to the database. In this case we will use `chinook.db` for the sake of easiness. The `models.py` will create an object relational model of the data using SQLAlchemy. `schema.py` will create a schema for our GraphQL API using graphene. And finally, a Flask web app of `app.py` that will wrap all of it. Let's make it one by one

## Database

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker

engine = create_engine('sqlite:///chinook.db', convert_unicode=True)
db_session = scoped_session(sessionmaker(autocommit=False,
                                         autoflush=False,
                                         bind=engine))
Base = declarative_base()
Base.query = db_session.query_property()

If you're confused of how it works, let me explain line by lines. 

In line #1-3, we imported the library we will use. Mostly are SQLAlchemy.  \
In line #5, we created a connection using SQLAlchemy engine. You can see thorough explanation on it's [documentatin page](https://docs.sqlalchemy.org/en/13/core/engines.html#custom-dbapi-connect-arguments). 

![](res/sqla_engine.png)

In line #6, we created a session, that holds all of our conversation with the databse.  

In line #9, we created a declarative base class of SQLAlchemy. SQLAlchemy object-relational configuration involves the combination of Table, mapper(), and class objects to define a mapped class. Declarative allows all three to be expressed at once within the class declaration. 

In line #10, we set our ORM base class query with the session ones.

## Models

This process is basically modelling our database in pythonic way. In order to model the database, we need to know the **database structure** first. Since we are using `chinook.db` from sqlite database sample, it comes with clear diagram structure as you can see below:
![](res/sqlite-sample-database-color.jpg)

I know, it's a bunch of tables. Let's try with one, the simple one, `artists` and `albums`. Let's create a Class of both of them, including their attributes. 

```python
from database import Base
from sqlalchemy import Column, DateTime, ForeignKey, Integer, String, func
from sqlalchemy.orm import backref, relationship

class Artist(Base):
    __tablename__ = 'artists'
    ArtistId = Column(Integer, primary_key=True)
    Name = Column(String)

class Album(Base):
    __tablename__ = 'albums'
    AlbumId = Column(Integer, primary_key=True)
    Title = Column(String)
    ArtistId = Column(Integer, ForeignKey('artists.ArtistId'))
```

Since the two tables are related, we need to create a relationship between the two. Take a look at the snippet

```python
class Artist(Base):
    __tablename__ = 'artists'
    ArtistId = Column(Integer, primary_key=True)
    Name = Column(String)
    Album = relationship("Album") # Relation to Album 

class Album(Base):
    __tablename__ = 'albums'
    AlbumId = Column(Integer, primary_key=True)
    Title = Column(String)
    ArtistId = Column(Integer, ForeignKey('artists.ArtistId'))
    Artist = relationship("Artist") # Relation to Artist
```    

## Schema

Now, once we already create a model of our tables `artists` anda `albums`, we need to create a schema. We will need our models first, so let's import `Artist` and `Album` class we've created in `models.py`. Let's start by create `schema.py` and import the dependencies.

**Note: Most of our job in GraphQL is defining the schema. So we will try to re-iterate this step later. For now, Let's just focus on building from the simple one**

```python 
from models import Artist as ArtistModel
from models import Album as AlbumModel
```

Next, we will need graphene and its sqlalchemy module to help us create a schema. So, let import the libraries
```python
import graphene
from graphene import relay
from graphene_sqlalchemy import SQLAlchemyConnectionField, SQLAlchemyObjectType
```

Now we're ready to build a schema. First, let's create an interface for our models so that they can match the 'graph' structure by wrap them into an object and set the interface into **GraphQL Node** . Add these classes into `schema.py`

```python 
class Artist(SQLAlchemyObjectType):
    class Meta:
        model = ArtistModel
        interfaces = (relay.Node, )

class Album(SQLAlchemyObjectType):
    class Meta:
        model = AlbumModel
        interfaces = (relay.Node, )
```

Now, you can say, that every instance of Artist and Albums as a Node (yes, just like a node in graph). And they are connected. Now let's define the query of what object the client can access

```python        
class Query(graphene.ObjectType):
    node = relay.Node.Field()
    
    allArtist = SQLAlchemyConnectionField(Artist, sort=Artist.sort_argument())
    allAlbum = SQLAlchemyConnectionField(Album, sort=None)
```

On the query above, we let the users to make use of `allArtist` and `allAlbum` queries. The sort arguments on `allArtist` meaning you can sort the result using Artist's attributes. We will try this later. The latest step is to wrap them all into a schema object :

```python
schema = graphene.Schema(query=Query, types=[Department, Employee, Role])
```

Here's the full code of `schema.py` right now

```python
from models import Artist as ArtistModel
from models import Album as AlbumModel

import graphene
from graphene import relay
from graphene_sqlalchemy import SQLAlchemyConnectionField, SQLAlchemyObjectType

class Artist(SQLAlchemyObjectType):
    class Meta:
        model = ArtistModel
        interfaces = (relay.Node, )

class Album(SQLAlchemyObjectType):
    class Meta:
        model = AlbumModel
        interfaces = (relay.Node, )

class Query(graphene.ObjectType):
    node = relay.Node.Field()
    allArtist = SQLAlchemyConnectionField(Artist, sort=Artist.sort_argument())
    allAlbum = SQLAlchemyConnectionField(Album, sort=None)

schema = graphene.Schema(query=Query, types=[Artist, Album])
```

## App

We already have what we need to create the API. Now, let's wrap them into one app using Flask. Create `app.py` and copy the following codes. 

```python 
from database import db_session, init_db
from flask import Flask
from schema import schema
from flask_graphql import GraphQLView

app = Flask(__name__)
app.debug = True

app.add_url_rule("/graphql", view_func=GraphQLView.as_view("graphql", schema=schema, graphiql=True))

@app.teardown_appcontext
def shutdown_session(exception=None):
    db_session.remove()

if __name__ == "__main__":
    app.run()
```

On the app above, set our graphql path on "/graphql". You can change this to whatever you want. We also imported `GraphQLView`to activate **graphiQL** to help us try the queries within the app. You can define where will the app hosted inside `app.run()`. By default, it will hosted in `localhost` with port `5000`

# Try the Queries

Open terminal, go to the directory and run `python app.py`. It will show where the app hosted and what port it used. Now go to your browser and hit `localhost:5000/graphql` and you should see an interface like this 
![](res/graphiql.png)

We previously define an allAlbum and allArtist query. Let's ther all Album first. 
type 
```JSON
{
    allAlbum
}```

on the graphql, hit `ctrl+enter` and it will automatically correct it for you from this one:
![](res/query1.png)

into this one:
![](res/query1result.png)

You might questing at first, what is edges and nodes. Here's how it works. We call the `allAlbum` query. All the results is stored as a graph-like object, connected each other onto one root node trough edge. If you want to access the value, you have to go trough edges and nodes. Here's the illustration.

![](res/graph_result.png)

Now let's try to take all albums with its artist

![](res/query2.png)

## Query Type
If you want, you can explicitly define your query types whether it's a `query` or `mutation`.
![](res/query2_type.png)

## Query Name

You can also specify a name for your query. This will help other person to understand, especially if you're working on team
![](res/query2_name.png)

## Sort the Query

Previosly we set the allArtist query to be sortable using `Artist.sort_arguments()`. Now let's try to get all artists  with their album names sorted by artist's name. You can use arguments `sort: ['attribute1_ASC', 'attribute2_DESC']`. You may add as many attributes as you want. the `ASC` and `DESC` just mean ascending or descending order. The graphiQL actually comes with code hints, mean while you're typing the query, you might get some hint like this, and it help! a lot ! especially if you're not familiar enough with the queries. This is an example of how it gives hints of what value you could add into the sort arguments.
![](res/query3_hint.png)

Let's get back to the business, here's how it result when we get all artist name and their album names sorte by artist name

![](res/query3.png)

If you're not using the graphiql UI and using Postman or curl, you can still see the sort arguments by checking the Model.sort_arguments()

## Query Filtering

Let's say we wanted only a specific artist. We can set a filter, let's say, using artistid.

Let's get back to the schema, and add an input of Artist Class

```python
from models import Artist as ArtistModel
from models import Album as AlbumModel

import graphene
from graphene import relay
from graphene_sqlalchemy import SQLAlchemyConnectionField, SQLAlchemyObjectType

class Artist(SQLAlchemyObjectType):
    class Meta:
        model = ArtistModel
        interfaces = (relay.Node, )

class Album(SQLAlchemyObjectType):
    class Meta:
        model = AlbumModel
        interfaces = (relay.Node, )

class Query(graphene.ObjectType):
    node = relay.Node.Field()
    allArtist = SQLAlchemyConnectionField(Artist, sort=Artist.sort_argument())
    allAlbum = SQLAlchemyConnectionField(Album, sort=None)
    
    # added this
    find_artist = graphene.Field(lambda: Artist, id=graphene.Int())
    def resolve_find_artist(self, info, **kwargs):
        query = Artist.get_query(info)
        id = kwargs.get("id")
        return query.filter(ArtistModel.ArtistId == id).first()

schema = graphene.Schema(query=Query, types=[Department, Employee, Role])
```

![](res/query4.png)

That's the basic of querying. What you need the rest is to :
- Model the whole database tables
- Add the recent model schemas

I have attached a sourcode that complete the whole tables (minus invoice))

# Send Query using Python

If you somehow create an application that make use of the API, here's an example of how to call it using python. First, we define the query

In [1]:
query = """
{
  allArtist{
    edges{
      node{
        Name
      }
    }
  }
}
"""

Then using builtin packages `requests`, we will try to send the query trough our API endpoint

In [4]:
import requests
import json 

url = 'http://localhost:5000/graphql'
r = requests.post(url, json={'query': query})
print(r.status_code)
print(r.json())

200
{'data': {'allArtist': {'edges': [{'node': {'Name': 'AC/DC'}}, {'node': {'Name': 'Accept'}}, {'node': {'Name': 'Aerosmith'}}, {'node': {'Name': 'Alanis Morissette'}}, {'node': {'Name': 'Alice In Chains'}}, {'node': {'Name': 'Antônio Carlos Jobim'}}, {'node': {'Name': 'Apocalyptica'}}, {'node': {'Name': 'Audioslave'}}, {'node': {'Name': 'BackBeat'}}, {'node': {'Name': 'Billy Cobham'}}, {'node': {'Name': 'Black Label Society'}}, {'node': {'Name': 'Black Sabbath'}}, {'node': {'Name': 'Body Count'}}, {'node': {'Name': 'Bruce Dickinson'}}, {'node': {'Name': 'Buddy Guy'}}, {'node': {'Name': 'Caetano Veloso'}}, {'node': {'Name': 'Chico Buarque'}}, {'node': {'Name': 'Chico Science & Nação Zumbi'}}, {'node': {'Name': 'Cidade Negra'}}, {'node': {'Name': 'Cláudio Zoli'}}, {'node': {'Name': 'Various Artists'}}, {'node': {'Name': 'Led Zeppelin'}}, {'node': {'Name': 'Frank Zappa & Captain Beefheart'}}, {'node': {'Name': 'Marcos Valle'}}, {'node': {'Name': 'Milton Nascimento & Bebeto'}}, {'node':

Then, using `Pandas`, we can normalize the result and store is as `DataFrame` data type. 

In [5]:
import pandas as pd
pd.io.json.json_normalize(r.json()['data']['allArtist']['edges'])

,node.Name
0,AC/DC
1,Accept
2,Aerosmith
3,Alanis Morissette
4,Alice In Chains
5,Antônio Carlos Jobim
6,Apocalyptica
7,Audioslave
8,BackBeat
9,Billy Cobham


# What's Next

We have created an API for user to query the data. Next we will make an API for user to Mutate the data in our database. 

    github: @iqbalbasyar 
    linkedin: iqbal basyar 